In [105]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import save_model
import os

classifier = Sequential()

#Convolution step
classifier.add(Convolution2D(32, 3, 3, input_shape= (64, 64, 3), activation='relu'))
classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#Convolution step
classifier.add(Convolution2D(64, 3, 3, activation='relu'))
classifier.add(Convolution2D(64, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#Flattening step
classifier.add(Flatten())
#Full connection (fully connected layer, hidden layer), create the classic ANN, Dense adds a fully connected layer
classifier.add(Dense(units= 128, activation='relu'))

#Output layer, if more than two outcomes use softmax
classifier.add(Dense(units= 1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/home/devinlax/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  if sys.path[0] == '':
/home/devinlax/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  del sys.path[0]
/home/devinlax/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/devinlax/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [106]:
cwd = '/home/devinlax/jupyter/Artificial_Neural_Networks/convolution/dataset/'

# Image augmentation to help prevent overfitten because 10000 images isn't very many
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        cwd + 'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        cwd + 'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [107]:
classifier.fit_generator(
        training_set,
        samples_per_epoch=8000,
        epochs=10,
        validation_data=test_set,
        nb_val_samples=2000,
        workers=8,
        use_multiprocessing=True)

/home/devinlax/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/devinlax/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., workers=8, use_multiprocessing=True, steps_per_epoch=250, validation_steps=2000)`
  
Epoch 1/10
250/250 [==============================] - 193s 771ms/step - loss: 0.6922 - accuracy: 0.5241 - val_loss: 0.6797 - val_accuracy: 0.5635
Epoch 2/10
250/250 [==============================] - 195s 778ms/step

In [77]:
classifier.save_weights('dog_cat_cnn.h5') # Save the model for later use
classifier.save('dog_cat_model.h5')


In [94]:

# how process individual images
import numpy as np
from keras.preprocessing import image
# classifier.load_weights('dog_cat_cnn.h5')
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    print('dog')
else:
    print('cat')

dog
